In [313]:
import datetime
import ipywidgets as widgets
import requests
from bs4 import BeautifulSoup
import json
import os
import pathlib
import time

In [314]:
from_page = 1
to_page = 2
link_file = pathlib.Path('./data/linklist.json')

In [315]:
known_link_ids = []
link_list = []
if os.path.exists(link_file):
    with open(link_file) as f:
        try:
            link_list = json.load(f)
            for linfo in link_list:
                known_link_ids.append(linfo['id'])
        except json.JSONDecodeError:
            print(f'Link file {link_file} is empty')
else:
    with open(link_file, 'w') as j:
        j.close()


In [316]:
i = from_page

has_results = True

while(has_results and i <= to_page):
    new_link_list = []  
    time.sleep(2)
    search_response = requests.get(f'https://magic.wizards.com/en/news/archive?search&page={i}&category=all&author=all&order=newest')
    page_result = search_response.text
    soup = BeautifulSoup(page_result, 'html.parser')
    
    # Check if page is empty:
    entry = soup.find(class_="css-36asz")
    if entry:
        print(f'Response from page {i} :'+entry.text.strip())
        has_results = False
        break
    
    entry_list = soup.find_all(class_="css-9f4rq")
    for entry in entry_list:
        parent = entry.findParent()
        link_id = parent.get('href')
        if not link_id in known_link_ids:
            link_info = {'id': link_id,
                        'link': 'https://magic.wizards.com'+link_id,
                        'name': parent.text,
                        'date_added': datetime.datetime.now().strftime("%Y-%m-%d, %H:%M:%S"),
                        'date_downloaded': ''
                        }
            print(f'Adding {link_id}')
            new_link_list.append(link_info)
            # print(f'LinkId {link_id} added')
        else:
            print(f'Link with Id {link_id} already exists')
        
    link_list.extend(new_link_list)
    if i % 10 == 0:
        print(f'Page: {i}')
        print(search_response.status_code)
    i+=1


with open(link_file, 'w') as j:
    json.dump(link_list, j, indent=4)





Link with Id /en/news/announcements/phyrexia-all-will-be-one-comprehensive-rules-changes already exists
Link with Id /en/news/announcements/phyrexia-all-will-be-one-update-bulletin already exists
Link with Id /en/news/mtg-arena/2023-mtg-arena-premier-play already exists
Link with Id /en/news/mtg-arena/the-most-played-cards-of-phyrexia-all-will-be-one-in-early-access already exists
Link with Id /en/news/mtg-arena/mtg-arena-announcements-february-6-2023 already exists
Link with Id /en/news/mtg-arena/mtg-arena-release-notes-phyrexia-all-will-be-one already exists
Link with Id /en/news/making-magic/mission-compleat-part-2 already exists
Link with Id /en/news/announcements/update-on-secret-lair-ssssssnakessssss-stonecoil-serpent-art already exists
Link with Id /en/news/feature/the-legends-of-phyrexia-all-will-be-one already exists
Link with Id /en/news/card-image-gallery/phyrexia-all-will-be-one-art-cards already exists
